In [2]:
!pip install pandas vadersentiment openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load data
df = pd.read_csv("/content/68b1acd44f393_Sample_Support_Emails_Dataset.csv")

# Filter support emails
keywords = ["support","help","query","request"]
df = df[df['subject'].str.lower().str.contains('|'.join(keywords)) |
         df['body'].str.lower().str.contains('|'.join(keywords))]

# Sentiment
analyzer = SentimentIntensityAnalyzer()
df["sentiment"] = df["body"].apply(lambda x:
    "Positive" if analyzer.polarity_scores(str(x))['compound'] > 0.05 else
    "Negative" if analyzer.polarity_scores(str(x))['compound'] < -0.05 else
    "Neutral"
)

# Priority
urgent_terms = ["urgent","immediate","critical","cannot access","down","blocked","billing error","charged twice"]
def get_priority(text):
    text = str(text).lower()
    return "Urgent" if any(term in text for term in urgent_terms) else "Not urgent"

df["priority"] = df["subject"] + " " + df["body"]
df["priority"] = df["priority"].apply(get_priority)

# Draft reply (simple rule-based)
def draft_reply(row):
    if row["priority"] == "Urgent":
        return f"Dear {row['sender']},\nWe understand your issue is urgent. Our team is prioritizing it and will resolve it shortly."
    else:
        return f"Dear {row['sender']},\nThank you for reaching out. We will get back to you with a solution soon."

df["draft_reply"] = df.apply(draft_reply, axis=1)

# Show results
df_display = df[["sender","subject","sentiment","priority","draft_reply"]]
df_display.head(10)


,sender,subject,sentiment,priority,draft_reply
0,eve@startup.io,Help required with account verification,Positive,Not urgent,"Dear eve@startup.io,\nThank you for reaching o..."
1,diana@client.co,General query about subscription,Positive,Not urgent,"Dear diana@client.co,\nThank you for reaching ..."
2,eve@startup.io,Immediate support needed for billing error,Positive,Urgent,"Dear eve@startup.io,\nWe understand your issue..."
3,alice@example.com,Urgent request: system access blocked,Positive,Urgent,"Dear alice@example.com,\nWe understand your is..."
5,alice@example.com,Critical help needed for downtime,Positive,Urgent,"Dear alice@example.com,\nWe understand your is..."
6,diana@client.co,Help required with account verification,Negative,Urgent,"Dear diana@client.co,\nWe understand your issu..."
7,diana@client.co,Support needed for login issue,Neutral,Not urgent,"Dear diana@client.co,\nThank you for reaching ..."
8,alice@example.com,General query about subscription,Neutral,Urgent,"Dear alice@example.com,\nWe understand your is..."
9,alice@example.com,Help required with account verification,Positive,Not urgent,"Dear alice@example.com,\nThank you for reachin..."
10,diana@client.co,Support needed for login issue,Positive,Not urgent,"Dear diana@client.co,\nThank you for reaching ..."
